In [419]:
import pandas as pd
import model_afolu as ma
import model_circular_economy as mc
import numpy as np
import os, os.path
import data_structures as ds
import setup_analysis as sa
import support_functions as sf
import importlib
import warnings
importlib.reload(ds)
importlib.reload(sa)


#import data_structures as ds
#importlib.reload(ds)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:489: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  key_dict = f"unit_energy_to_energy_equivalent_{me}"
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:489: UserWarning: Invalid subsector attribute 'key_varreqs_all'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector'
  key_dict = f"unit_energy_to_energy_equivalent_{me}"
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:489: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector'
  key_dict = f"unit_energy_to_energy_equivalent_{me}"
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structure

<module 'setup_analysis' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/setup_analysis.py'>

In [301]:
#model_ce = mc.CircularEconomy(sa.model_attributes)
model_afolu = ma.AFOLU(sa.model_attributes)


In [396]:
sa.model_attributes.get_baseline_scenario_id(sa.model_attributes.dim_strategy_id)

0

In [386]:
#sa.model_attributes.dict_attributes["dim_time_series_id"].table("dime")
sa.model_attributes.all_dims

def get_baseline_scenario_id(dim: str):
    
    if dim not in sa.model_attributes.all_dims:
        fpl = sf.format_print_list(sa.model_attributes.all_dims)
        raise ValueError(f"Invalid dimension '{dim}': valid dimensions are {fpl}.")
        
    # get field to check
    field_check = f"baseline_{dim}"
    if field_check not in sa.model_attributes.dict_attributes[f"dim_{dim}"].table:
        warnings.warn(f"No baseline specified for dimension '{dim}'.")
        return None
    else:
        tab = sa.model_attributes.dict_attributes[f"dim_{dim}"].table
        tab_red = list(tab[tab[field_check] == 1][dim])
        
        if len(tab_red) > 1:
            raise ValueError(f"Multiple baselines specified for dimension {dim}. Ensure that only baseline is set in the attribute table at '{tab.fp_table}'")
        
        return tab_red[0]
        
    
get_baseline_scenario_id(sa.model_attributes.dim_strategy_id)
        

0

In [332]:
# get the
tp = max(sa.model_attributes.get_time_periods()[0])
f0, f1 = (f"min_{tp}", f"max_{tp}")
df_template_afolu = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "tmp_afolu_vars_with_mcmc_new.csv"))
df_template_afolu.drop([f0, f1], axis = 1, inplace = True)

df_sampling_range_defaults = sa.model_attributes.build_default_sampling_range_df()

df_template_afolu = pd.merge(df_template_afolu, df_sampling_range_defaults, how = "left", on = ["variable"])
df_template_afolu = df_template_afolu.dropna(subset = ["max_35", "min_35"], how = "any")
df_template_afolu.drop(["src"], axis = 1, inplace = True)

df_template_afolu["time_series_id"] = np.zeros(len(df_template_afolu)).astype(int)
df_template_afolu["strategy_id"] = np.zeros(len(df_template_afolu)).astype(int)
df_template_afolu["uniform_scaling_q"].iloc[~df_template_afolu["uniform_scaling_q"].isna()] = df_template_afolu["uniform_scaling_q"].iloc[2]

# order
tp_max = max(sa.model_attributes.get_time_periods()[0])
fields_minmax = [f"min_{tp_max}", f"max_{tp_max}"]
fields_prep = [x for x in df_template_afolu.columns if not x.isnumeric() and (x not in fields_minmax)] + fields_minmax
fields_times = [x for x in df_template_afolu.columns if (x not in fields_prep)]

df_template_afolu = df_template_afolu[fields_prep + fields_times]
subsectors = [sa.model_attributes.dict_model_variable_to_subsector[sa.model_attributes.dict_variables_to_model_variables[x]] for x in df_template_afolu["variable"]]
df_template_afolu["subsector"] = subsectors

for sec in ["AFOLU", "Socioeconomic"]:

    vars_keep = sa.model_attributes.get_variables_by_sector(sec, "input")
    df_exp = df_template_afolu[df_template_afolu["variable"].isin(vars_keep)].copy().reset_index(drop = True).drop_duplicates().sort_values(by = ["subsector", "variable"])
    
    dict_out = {
        "strategy_id-0": df_exp, 
        "strategy_id-1": df_exp.iloc[0:0]
    }
    sf.dict_to_excel(sa.excel_template_path(sec, "arg", "demo", True), dict_out)



/Users/jsyme/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:462: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


In [335]:
##############################
#    BUILD WASTE TEMPLATE    #
##############################

sec = "Circular Economy"
l_vars = sa.model_attributes.get_variables_by_sector(sec, "input")
l_vars.sort()

df_base = df_exp.iloc[0:0].copy()
df_base["variable"] = l_vars
# add subsectors
subsectors = [sa.model_attributes.dict_model_variable_to_subsector[sa.model_attributes.dict_variables_to_model_variables[x]] for x in df_base["variable"]]
df_base["subsector"] = subsectors
df_base.drop(["min_35", "max_35"], axis = 1, inplace = True)
df_base = pd.merge(df_base, df_sampling_range_defaults)

df_safe = pd.read_excel(sa.excel_template_path(sec, "arg", "demo", True).replace("_demo.xlsx", "_demo_safe.xlsx"))
dft_out = pd.merge(df_base[[x for x in df_base.columns if (x != "0")]], df_safe[["variable", "0"]].dropna(subset = ["0"]).reset_index(drop = True), how = "left")
dft_out = dft_out[df_template_afolu.columns]

if True:
    


    dict_out = {
        "strategy_id-0": dft_out, 
        "strategy_id-1": dft_out.iloc[0:0]
    }
    sf.dict_to_excel(sa.excel_template_path(sec, "arg", "demo", True), dict_out)

    

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:910: UserWarning: clean_partial_category_dictionary: Invalid categories values 'domestic_rural_ww', 'domestic_urban_ww' dropped when cleaning the dictionary. Category values not found.
  warnings.warn(f"clean_partial_category_dictionary: Invalid categories values {missing_vals} dropped when cleaning the dictionary. Category values not found.")


In [488]:
# expand later to integrate strategy dimension
def build_modvar_input_db_from_templates(
    model_attributes: ds.ModelAttributes,
    sectors: list, 
    region: str, 
    template_type: str,
    repl_missing_with_base: bool = True
) -> pd.DataFrame:
    
    ##  run some checksee
    
    # check region
    region = region.lower()
    if region not in model_attributes.dict_attributes["region"].key_values:
        valid_regions = sf.format_print_list(model_attributes.dict_attributes["region"].key_values)
        raise ValueError(f"Invalid region '{region}' specified. Valid regions are {valid_regions}")
        
    # check sectors
    sectors_drop = [x for x in sectors if (x not in model_attributes.all_sectors)]
    if len(sectors_drop) > 0:
        secs_drop = sf.format_print_list(sectors_drop)
        raise ValueError(f"Invalid sectors {secs_drop} found. Valid sectors are {model_attributes.all_sectors}.")
    
    
    ##
    ##  TEMP: 0 only
    ##
    strat_base = 0
    sheet_base = f"{model_attributes.dim_strategy_id}-{strat_base}"
    
    strats_eval = [strat_base]
    
    df_out = []
    
        
    for sec in sectors:
        fp_templ = sa.excel_template_path(sec, region, template_type, True)
        if not os.path.exists(fp_templ):
            raise ValueError(f"Error: path '{fp_templ}' to template not found.")
        # check available sheets and ensure baseline is available
        sheets_avail = pd.ExcelFile(fp_templ).sheet_names
        if sheet_base not in sheets_avail:
             raise ValueError(f"Baseline strategy sheet {sheet_base} not found in '{fp_templ}'. The template must have a sheet for the baseline strategy.")
        
        
        for strat in strats_eval:
            sheet = f"{model_attributes.dim_strategy_id}-{strat}"
            if not sheet in sheets_avail:
                msg = f"Sheet {sheet} not found in '{fp_templ}'. Check the template."
                if repl_missing_with_base:
                    warnings.warn(f"{msg}. The baseline strategy will be used.")
                    sheet = sheet_base
                else:
                    raise ValueError(msg)
            
            # 
            df_tmp = pd.read_excel(fp_templ, sheet_name = sheet)
            df_tmp[model_attributes.dim_strategy_id] = strat
            
            #
            #   ADD CHECKS FOR TIME PERIODS
            #
            
            
            #
            #   ADD DIFFERENT STEPS FOR NON-BASELINE STRATEGY
            #
            
            if len(df_out) == 0:
                df_out.append(df_tmp)
            else:
                df_out.append(df_tmp[df_out[0].columns])
                
    df_out = pd.concat(df_out, axis = 0).sort_values(by = ["subsector", "variable"]).reset_index(drop = True)
        
    return df_out
        
# function to convert a model variable input database into a simple projection input dataframe
def build_basic_df_from_modvar_inputs(
    model_attributes: ds.ModelAttributes, 
    df_mv: pd.DataFrame
) -> pd.DataFrame:
    """
        build_basic_df_from_modvar_inputs will take a model input database and transform it (under baseline assumptions) into a projection dataframe.

        - model_attributes: a ModelAttributes class

        - df_mv: a dataframe of model variables
    """

    df_mv_out = df_mv[[str(x) for x in model_attributes.get_time_periods()[0]]].transpose().reset_index(drop = True)
    var_fields = list(df_mv["variable"])
    df_mv_out.rename(columns = dict(zip([(x) for x in range(len(df_mv_out.columns))], var_fields)), inplace = True)

    df_mv_out[model_attributes.dim_time_period] = list(range(len(df_mv_out)))
    var_fields.sort()

    return df_mv_out[[model_attributes.dim_time_period] + var_fields]

        
df_mv_tmp = build_modvar_input_db_from_templates(sa.model_attributes, ["Socioeconomic", "Circular Economy", "AFOLU"], "arg", "demo")       
df_mv_out = build_basic_df_from_modvar_inputs(sa.model_attributes, df_mv_tmp)
df_mv_out.to_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"), index = None, encoding = "UTF-8")



In [422]:
[x for x in df_mv_out.columns if "trww" in x]

['ef_trww_treated_aerobic_g_n2o_per_g_n',
 'ef_trww_treated_anaerobic_g_n2o_per_g_n',
 'ef_trww_treated_latrine_improved_g_n2o_per_g_n',
 'ef_trww_treated_latrine_unimproved_g_n2o_per_g_n',
 'ef_trww_treated_septic_g_n2o_per_g_n',
 'ef_trww_untreated_no_sewerage_g_n2o_per_g_n',
 'ef_trww_untreated_with_sewerage_g_n2o_per_g_n',
 'frac_trww_tow_removed_treated_aerobic',
 'frac_trww_tow_removed_treated_anaerobic',
 'frac_trww_tow_removed_treated_latrine_improved',
 'frac_trww_tow_removed_treated_latrine_unimproved',
 'frac_trww_tow_removed_treated_septic',
 'frac_trww_tow_removed_untreated_no_sewerage',
 'frac_trww_tow_removed_untreated_with_sewerage',
 'mcf_trww_treated_aerobic',
 'mcf_trww_treated_anaerobic',
 'mcf_trww_treated_latrine_improved',
 'mcf_trww_treated_latrine_unimproved',
 'mcf_trww_treated_septic',
 'mcf_trww_untreated_no_sewerage',
 'mcf_trww_untreated_with_sewerage']

In [336]:
set(df_safe["variable"]) - set(dft_out["variable"])
l = list(set(dft_out["variable"]) - set(df_safe["variable"]))
for k in l:
    print(k)

In [307]:
df_safe = pd.read_excel(sa.excel_template_path(sec, "arg", "demo", True).replace("_demo.xlsx", "_demo_safe.xlsx"))
#dft = dict_out["strategy_id-0"]
#dft_out = pd.merge(dft[[x for x in dft.columns if (x != "0")]], df_safe[["variable", "0"]].dropna(subset = ["0"]).reset_index(drop = True), how = "left")



In [437]:
dft = sa.model_attributes.dict_attributes["region"].table.copy()
dft["region"] = [x.upper() for x in dft["region"]]

df_wb_waste = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/waste_management_data/wb_whatawaste_country_level_data_0.csv")
df_wb_waste = pd.merge(df_wb_waste, dft[["region"]], left_on = ["iso3c"], right_on = ["region"], how = "inner")
[x for x in df_wb_waste.columns if "recy" in x]

np.mean(df_wb_waste["waste_treatment_recycling_percent"].dropna())


6.613846153846154

In [485]:
arr = np.array(df_wb_waste[["total_msw_total_msw_generated_tons_year", "population_population_number_of_people"]])#.columns
np.dot(arr[:, 0], arr[:, 1])/(sum(arr[:, 1])*sum(arr[:, 0]))
               
sum((arr[:, 1]*(arr[:, 0]/arr[:, 1]))/sum(arr[:, 1]))



ind_waste = np.array(df_wb_waste[[x for x in df_wb_waste.columns if "special_waste_industrial_waste_tons_year" in x]].fillna(0).sum(axis = 1))
gdp = (np.array(df_wb_waste["gdp"])/1000)
np.mean(ind_waste/gdp)




0.0

In [ ]:
### importlib.reload(sf)
importlib.reload(ds)
importlib.reload(sa)
sa.excel_template_path("AFOLU", "arg", "demo", True)

In [360]:

?pd.DataFrame.to_excel

Signature:
pd.DataFrame.to_excel(
    self,
    excel_writer,
    sheet_name='Sheet1',
    na_rep='',
    float_format=None,
    columns=None,
    header=True,
    index=True,
    index_label=None,
    startrow=0,
    startcol=0,
    engine=None,
    merge_cells=True,
    encoding=None,
    inf_rep='inf',
    verbose=True,
    freeze_panes=None,
) -> None
Docstring:
Write object to an Excel sheet.

To write a single object to an Excel .xlsx file it is only necessary to
specify a target file name. To write to multiple sheets it is necessary to
create an `ExcelWriter` object with a target file name, and specify a sheet
in the file to write to.

Multiple sheets may be written to by specifying unique `sheet_name`.
With all data written to the file it is necessary to save the changes.
Note that creating an `ExcelWriter` object with a file name that already
exists will result in the contents of the existing file being erased.

Parameters
----------
excel_writer : str or ExcelWriter object
  

In [335]:
sa.model_attributes.get_sector_attribute("AFOLU", "abbreviation_sector")

'af'

In [131]:
os.listdir(sa.dir_proj)

['ref',
 '.DS_Store',
 'LICENSE',
 'project_metadata.json',
 'python',
 'out',
 'pyproject.toml',
 'docs',
 'model_overview.md',
 'README.md',
 '.gitignore',
 'julia',
 'README.rst',
 '.git']